# 고급 ufunc 사용법

In [1]:
# 많은 NumPy 사용자는 유니버셜 함수로 제공되는 빠른 원소별 연산만을 주로 사용한다.
# 반복문을 작성하지 않고 좀 더 간결한 코드를 작성할 수 있는 다양한 부가적인 기능이 있다.

    - A.4.1 ufunc 인스턴스 메서드

In [2]:
# NumPy의 이항 ufunc는 그런 특수한 벡터 연산을 수행하기 위한 특수한 메서드를 제공한다.
# 이 메서드를 [표 A-2]에 정리해두었지만, 어떻게 동작하는지 몇 가지 예제를 통해 익혀보자.

In [3]:
# reduce는 하나의 배열을 받아서 순차적인 이항 연산을 통해 축에 따라 그 값을 집계해준다.
# 예를 들어 배열의 모든 원소를 더하는 방법으로 np.add.reduce를 사용할 수 있다.

In [5]:
import numpy as np
arr = np.arange(10)

In [6]:
np.add.reduce(arr)

45

In [7]:
arr.sum()

45

In [8]:
# 시작값(add에서는 0)은 ufunc에 의존적이다.
# 만약 axis 인자가 넘어오면 reduce는 그 축을 따라 수행된다.
# 이를 통해 축약된 방법으로 어떤 질문에 대한 답을 구할 수 있다.
# 약간 복잡한 예제로 np.logical_and를 사용해서 배열의 각 로우에 있는 값이 정렬된 상태인지 검사하는 것을 생각해볼 수 있다.

In [9]:
np.random.seed(12346) # 동일한 난수 발생을 위해 시드값 직접 지정

In [10]:
arr = np.random.randn(5, 5)

In [11]:
arr[::2].sort(1) # 일부 로우를 정렬

In [12]:
arr[:, :-1] < arr[:, 1:]

array([[ True,  True,  True,  True],
       [False,  True, False, False],
       [ True,  True,  True,  True],
       [ True, False,  True,  True],
       [ True,  True,  True,  True]])

In [13]:
np.logical_and.reduce(arr[:, :-1] < arr[:, 1:], axis=1)

array([ True, False,  True, False,  True])

In [14]:
# logical_and.reduce는 all 메서드와 동일하다.

In [15]:
# cumsum 메서드가 sum 메서드와 관련 있는 것처럼 accumulate는 reduce 메서드와 관련있다.

In [16]:
# accumulate 메서드는 누계를 담고 있는 같은 크기의 배열을 생성한다.

In [17]:
arr = np.arange(15).reshape((3, 5))

In [18]:
np.add.accumulate(arr, axis=1)

array([[ 0,  1,  3,  6, 10],
       [ 5, 11, 18, 26, 35],
       [10, 21, 33, 46, 60]])

In [19]:
# outer 메서드는 두 배열 간의 벡터곱(외적)을 계산한다.

In [20]:
arr = np.arange(3).repeat([1, 2, 2])

In [21]:
arr

array([0, 1, 1, 2, 2])

In [22]:
np.multiply.outer(arr, np.arange(5))

array([[0, 0, 0, 0, 0],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 2, 4, 6, 8],
       [0, 2, 4, 6, 8]])

In [23]:
# outer 메서드 결과의 차원은 입력한 차원의 합이 된다.

In [24]:
x, y = np.random.randn(3, 4), np.random.randn(5)

In [25]:
result = np.subtract.outer(x, y)

In [26]:
result.shape

(3, 4, 5)

In [27]:
# 마지막으로 reduceat 메서드는 로컬 reduce를 수행하는데, 본질적으로 로컬 reduce는 배열의 groupby 연산으로 배열의 슬라이스를 모두 집계하는 것이다.
# pandas의 GroupBy 기능보다 유연하지 않지만 적절한 상황에서 사용한다면 매우 빠르고 강력한 메서드다.
# reduceat 메서드는 값을 어떻게 나누고 집계할지 나타내는 경계 목록을 인자로 받는다.

In [28]:
arr = np.arange(10)

In [29]:
np.add.reduceat(arr, [0, 5, 8])

array([10, 18, 17])

In [30]:
# 이 결과는 arr[0:5], arr[5:8], arr[8:]에 대한 수행결과(여기서는 합)다.
# 다른 메서드와 마찬가지로 axis 인자를 넘겨줄 수 있다.

In [31]:
arr = np.multiply.outer(np.arange(4), np.arange(5))

In [32]:
arr

array([[ 0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4],
       [ 0,  2,  4,  6,  8],
       [ 0,  3,  6,  9, 12]])

In [33]:
np.add.reduceat(arr, [0, 2, 4], axis=1)

array([[ 0,  0,  0],
       [ 1,  5,  4],
       [ 2, 10,  8],
       [ 3, 15, 12]])

In [35]:
# [표 A-2]에 일부 ufunc 메서드를 나열해두었다. 페이지 610

# A.4.2 파이썬으로 사용자 정의 ufunc 작성하기

In [36]:
# ufunc와 유사한 사용자 함수를 만들 수 있는 몇 가지 기능이 있다.
# 일반적인 방법은 NumPy의 C API를 이용하는 것이지만 이 책에서 다룰 내용은 아니므로 여기서는 순수 파이썬 ufunc만 살펴보겠다.

In [37]:
# numpy.frompyfunc는 입력과 출력에 대한 표준과 함께 파이썬 함수를 인자로 취한다.
# 예를 들어 원소별로 합을 구하는 함수는 다음과 같이 작성할 수 있다.

In [38]:
def add_elements(x, y):
    return x + y

In [39]:
add_them = np.frompyfunc(add_elements, 2, 1)

In [40]:
add_them(np.arange(8), np.arange(8))

array([0, 2, 4, 6, 8, 10, 12, 14], dtype=object)

In [41]:
# frompyfunc를 이용해서 생성한 함수는 항상 파이썬 객체가 담긴 배열을 반환하는데 이는 그다지 유용하지 못하다.
# 다행스럽게도 대안이 있는데 numpy.vectorize을 사용하면 반환 자료형을 지정할 수 있는 이점이 있다.

In [42]:
add_them = np.vectorize(add_elements, otypes=[np.float64])

In [43]:
add_them(np.arange(8), np.arange(8))

array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14.])

In [44]:
# 이 두 함수는 ufunc 스타일의 함수를 만드는 방법을 제공한다.
# 하지만 각 원소를 계산하기 위해 파이썬 함수를 호출하게 되므로 NumPy의 C 기반 ufunc 반복문보다 많이 느리다.

In [45]:
arr = np.random.randn(10000)

In [46]:
%timeit add_them(arr, arr)

1.5 ms ± 40.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [47]:
%timeit np.add(arr, arr)

2.84 µs ± 181 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [48]:
# A.7절에서는 Numba(눔바) 프로젝트를 이용하여 파이썬에서 빠르게 동작하는 ufunc를 작성하는 방법을 살펴보겠다.